# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
import numpy as np
#from sql_queries import *
from datetime import datetime

In [2]:
#cur.execute("DROP DATABASE IF EXISTS sparkifydb")
#cur.execute("CREATE DATABASE sparkifydb WITH ENCODING 'utf8' TEMPLATE template0")
#conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
#cur = conn.cursor()

 # connect to default database
conn = psycopg2.connect("host=127.0.0.1 dbname=studentdb user=student password=student")
conn.set_session(autocommit=True)
cur = conn.cursor()
    
    # create sparkify database with UTF8 encoding
cur.execute("DROP DATABASE IF EXISTS sparkifydb")
cur.execute("CREATE DATABASE sparkifydb WITH ENCODING 'utf8' TEMPLATE template0")

    # close connection to default database
conn.close()    
    
    # connect to sparkify database
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()
    

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('data/song_data/A')


In [5]:
filepath = song_files[0]

In [6]:
#Read JSON file through PANDAS function read_json
data = pd.read_json(filepath,lines=True)
print(data)

            artist_id  artist_latitude  artist_location  artist_longitude  \
0  ARD7TVE1187B99BFB1              NaN  California - LA               NaN   

  artist_name   duration  num_songs             song_id             title  \
0      Casual  218.93179          1  SOMZWCG12A8C13C480  I Didn't Mean To   

   year  
0     0  


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
#Convert data to a Data Frame for easy execution & print values
df = pd.DataFrame(data)
print(df)

            artist_id  artist_latitude  artist_location  artist_longitude  \
0  ARD7TVE1187B99BFB1              NaN  California - LA               NaN   

  artist_name   duration  num_songs             song_id             title  \
0      Casual  218.93179          1  SOMZWCG12A8C13C480  I Didn't Mean To   

   year  
0     0  


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
#For test purposes only, CREATE table
try:
    cur.execute("CREATE TABLE IF NOT EXISTS songs2(songplay_id varchar, title varchar, artist_id varchar, year int, duration float)")
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)
    
#Insert dataframe data using function ITERTUPLES() in a for loop
try:
     for row in df.itertuples():
         cur.execute("INSERT INTO songs2(songplay_id, title, artist_id, year, duration)\
                      VALUES (%s,%s,%s,%s,%s)",\
                      (row.song_id,
                       row.title,
                       row.artist_id,
                       row.year,
                       row.duration)
                   ) 

except psycopg2.Error as e: 
    print("Error: INsert error")
    print (e)


conn.commit()

#Check if values have been correctly inserted & print them out
cur.execute("SELECT * FROM songs2;")
row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()


('SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179)


Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
#For test purposes only, CREATE table
try:
    cur.execute("CREATE TABLE IF NOT EXISTS artists(artist_id varchar,name text, location varchar,latitude float, longitude float)")
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)
    
#Insert dataframe data using function ITERTUPLES() in a for loop
try:
     for row in df.itertuples():
         cur.execute("INSERT INTO artists(artist_id,name,location,latitude,longitude)\
                      VALUES (%s,%s,%s,%s,%s)",\
                      (row.artist_id,
                       row.artist_name,
                       row.artist_location,
                       row.artist_latitude,
                       row.artist_longitude)
                   ) 

except psycopg2.Error as e: 
    print("Error: INsert error")
    print (e)


conn.commit()

#Check if values have been correctly inserted & print them out
cur.execute("SELECT * FROM artists;")
row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()


('ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan)


#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [10]:
#Get all the files from the log folder
log_files = get_files('data/log_data/2018/11')


In [11]:
#Pick up the first file in the log folder
filepath = log_files[0]

In [30]:
#Using PANDA functions, read through the contents of the first file and display that
df_log = pd.read_json(filepath, lines=True)
df_log.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
3,Maroon 5,Logged In,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Won't Go Home Without You,200,1543540625796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Train,Logged In,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Hey_ Soul Sister,200,1543540856796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [33]:
#Filter records by NextSong action & store it in a variable df_filter
df_filter = df_log[(df_log['page'] == 'NextSong')]


In [34]:
#Convert data in ts column to date and time. Create another column timestamp and add that to the dataframe.    
df_filter['timestamp'] = (pd.to_datetime(df_filter['ts'], unit='ms')
                              .dt.tz_localize(None))
#converted_df = pd.to_datetime(df_filter['ts'], unit='ms')
#df_filter['timestamp'] = converted_df
#print(converted_df)
print(df_filter)


                                                artist       auth firstName  \
0                                        Stephen Lynch  Logged In    Jayden   
1                                              Manowar  Logged In     Jacob   
2                                            Morcheeba  Logged In     Jacob   
3                                             Maroon 5  Logged In     Jacob   
4                                                Train  Logged In     Jacob   
5                                                LMFAO  Logged In     Jacob   
6                                             DJ Dizzy  Logged In     Jacob   
7                              Fish Go Deep & Tracey K  Logged In     Jacob   
9                                                  M83  Logged In     Jacob   
10                                              Saybia  Logged In     Jacob   
11                                       Local Natives  Logged In     Jacob   
12                                          South Pa

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [ ]:
time_data = ()
column_labels = ()

In [ ]:
time_df = 
time_df.head()

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [ ]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [ ]:
user_df = 

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [ ]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = ()
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.